# Customer Churn

## Table of Contents

- [Problem Statement](#Problem-Statement)
- [Imports](#Imports)
- [Data Loading](#Data-Loading)
- [Exploratory Data Analysis (EDA)](#Exploratory-Data-Analysis-\(EDA\))
  - [Data Overview](#Data-Overview)
  - [Null/NaN Values](#Null/NaN-Values)
  - [Data Visualization](#Data-Visualization)
- [Preliminary Baseline Model](#Preliminary-Baseline-Model)
  - [Data Preprocessing](#Data-Preprocessing)
  - [Modeling](#Modeling)
- [Feature Engineering and Selection](#Feature-Engineering-and-Selection)
  - [Feature Importance Analysis](#Feature-Importance-Analysis)
  - [Feature Selection](#Feature-Selection)
  - [Dimensionality Reduction](#Dimensionality-Reduction)
- [Handling Class Imbalance (if any)](#Handling-Class-Imbalance-\(if-any\))
- [Model Comparisons](#Model-Comparisons)
- [Model Tuning](#Model-Tuning)
- [Model Evaluation](#Model-Evaluation)
- [Conclusion](#Conclusion)

# Problem Statement

# Imports

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# Data Loading

In [2]:
os.chdir('..')

In [3]:
cust_churn = pd.read_excel('data/CustomerChurn.xlsx')
demo = pd.read_excel('data/Telco_customer_churn_demographics.xlsx')
location = pd.read_excel('data/Telco_customer_churn_location.xlsx')
population = pd.read_excel('data/Telco_customer_churn_population.xlsx')
services = pd.read_excel('data/Telco_customer_churn_services.xlsx')
status = pd.read_excel('data/Telco_customer_churn_status.xlsx')
churn = pd.read_excel('data/Telco_customer_churn.xlsx')

# Exploratory Data Analysis (EDA)

## Data Overview

## Null/NaN Values

## Data Visualization

# Preliminary Baseline Model

## Data Preprocessing

## Modeling

# Feature Engineering and Selection

## Feature Importance Analysis

## Feature Selection

## Dimensionality Reduction

# Handling Class Imbalance

# Model Comparisons

# Model Tuning

# Model Evaluation

# Conclusion